In [344]:
# 1. 모듈 임포트
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver import Keys
import requests

# 폴더 관리를 위한 모듈(5번 항목)
import os

# 이미지 링크 주소 파싱을 위한 모듈(6번 항목)
from urllib.parse import urlparse, unquote

In [345]:
# 2. 드라이버 옵션 설정 및 브라우저 띄우기
chrom_options = webdriver.ChromeOptions()
chrom_options.add_experimental_option('detach', True)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), 
                          options=chrom_options)
time.sleep(1)

In [346]:
# 3-1. 구글 이미지 검색 주소 이동
url = "https://www.google.com/imghp?hl=ko&tab=wi"
driver.get(url)
time.sleep(2)

In [347]:
# 3-2. 검색어 입력 및 검색
img_name = "마동석"
search_img = driver.find_element(By.CSS_SELECTOR, "textarea.gLFyf")
search_img.send_keys(img_name)
time.sleep(1)
search_img.send_keys(Keys.ENTER)

In [348]:
# 4-1. 처음 이미지 선택(선택해서 클릭한 이미지가 원본 이미지)
first_img = driver.find_element(By.CSS_SELECTOR, "img.rg_i.Q4LuWd")
first_img.click()
time.sleep(1)

In [349]:
# 4-2. 원본 이미지 링크 확인
target_img = driver.find_element(By.CSS_SELECTOR, "img.sFlh5c.pT0Scc.iPVvYb")
target_img_link = target_img.get_attribute("src")
print(target_img_link)

https://pds.joongang.co.kr/news/component/htmlphoto_mmdata/201608/04/htm_2016080484837486184.jpg


In [363]:
# 5. 목표 사진 개수 설정 및 저장 폴더 설정
target_count = 10
save_dir = f"./{img_name}/"

if not os.path.exists(save_dir):
    os.makedirs(save_dir, exist_ok=True)

print(f"'{save_dir}' 폴더가 생성되었거나 이미 존재합니다.")

'./마동석/' 폴더가 생성되었거나 이미 존재합니다.


In [356]:
# 6-1. 다음 이미지 버튼 요소 추출 테스트
next_div = driver.find_elements(By.CSS_SELECTOR, "div.HJRshd")
# 처음 이미지 일때는 2개이며, 원하는 요소는 0번째 
# 다음 이미지일 때는 버튼이 3개, 원하는 요소는 1번째 따라서 인덱스는 뒤에서 -2
target_div = next_div[-2]

In [357]:
# 6-2. 버튼 테스트
buttons = target_div.find_elements(By.TAG_NAME, "button")
# 버튼 확인용
for btn in buttons:
    print(btn.get_attribute("aria-label"))

이전 이미지
다음 이미지
이 검색 결과에 관한 작업 더보기
닫기


In [361]:
# 6-3. 이미지 저장 링크 테스트
target_img = driver.find_elements(By.CSS_SELECTOR, "img.sFlh5c.pT0Scc.iPVvYb")
for img in target_img:
    print("이미지 링크입니다", img.get_attribute("src"))

이미지 링크입니다 https://pds.joongang.co.kr/news/component/htmlphoto_mmdata/201608/04/htm_2016080484837486184.jpg
이미지 링크입니다 https://upload.wikimedia.org/wikipedia/commons/thumb/1/1f/Ma_Dong-seok.png/250px-Ma_Dong-seok.png


In [362]:
# 6-4 이미지 저장 테스트 - 이미지 데이터를 GET 요청으로 가져오기
target_img_link = target_img[-1].get_attribute("src")
response = requests.get(target_img_link)

# 요청이 성공적인 경우, 이미지를 로컬 파일로 저장
if response.status_code == 200:
    # 이미지 확장자 확인 및 쿼리 파라미터 제거를 위한 작업
    path = urlparse(target_img_link).path
    decoded_path = unquote(path)
    file_name = decoded_path.split("/")[-1]
    file_ext = file_name.split(".")[-1]
    save_name = f"{save_dir}{img_name}_test.{file_ext}"
    # 바이너리 쓰기 모드로 파일 열기
    with open(save_name, 'wb') as file:
        file.write(response.content)
    print(f"이미지가 성공적으로 다운로드되었습니다: {save_name}")
else:
    print("이미지 다운로드 실패")

이미지가 성공적으로 다운로드되었습니다: ./마동석/마동석_test.png


In [374]:
# 6-5. 최종 : 다음 이미지 클릭하며, 원본 이미지 저장
for count in range(target_count):
    # 다음 이미지 버튼 요소 가져오기
    next_div = driver.find_elements(By.CSS_SELECTOR, "div.HJRshd")
    target_div = next_div[-2] 

    # 다음 이미지 버튼 클릭
    buttons = target_div.find_elements(By.TAG_NAME, "button")
    next_btn = buttons[1]
    next_btn.click() # 다음 이미지 버튼 클릭

    # target 이미지 선택
    target_img = driver.find_elements(By.CSS_SELECTOR, "img.sFlh5c.pT0Scc.iPVvYb")[-1]
    target_img_link = target_img.get_attribute("src")
    print(target_img_link)

    # 이미지 데이터를 GET 요청으로 가져오기 - header 정보를 넣어 주는 것이 좋습니다.
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.get(target_img_link, headers=headers)
    # 요청이 성공적인 경우, 이미지를 로컬 파일로 저장
    if response.status_code == 200:
        # 이미지 확장자 확인 및 쿼리 파라미터 제거를 위한 작업
        path = urlparse(target_img_link).path
        decoded_path = unquote(path)
        file_name = decoded_path.split("/")[-1]
        file_ext = file_name.split(".")[-1]
        save_name = f"{save_dir}{img_name}_{count}.{file_ext}"
        # 바이너리 쓰기 모드로 파일 열기
        with open(save_name, 'wb') as file:
            file.write(response.content)
        print(f"이미지가 성공적으로 다운로드되었습니다: {save_name}")
    else:
        print(f"{response.status_code}_이미지 다운로드 실패")
    time.sleep(1)

https://upload.wikimedia.org/wikipedia/commons/thumb/1/1f/Ma_Dong-seok.png/250px-Ma_Dong-seok.png
이미지가 성공적으로 다운로드되었습니다: ./마동석/마동석_0.png
https://i.namu.wiki/i/Pkn5aM5-AsBHliHLNmpgQGNfePQSdIwIy1L52KVo_o3biJNKCyuqRvRt1o6l8WrWJgH5VNSZC28QXwZp_b990w.webp
이미지가 성공적으로 다운로드되었습니다: ./마동석/마동석_1.webp
https://pds.joongang.co.kr/news/component/htmlphoto_mmdata/201912/15/07712b9e-e451-49c7-a65c-fb94b6dcda0b.jpg
이미지가 성공적으로 다운로드되었습니다: ./마동석/마동석_2.jpg
https://file2.nocutnews.co.kr/newsroom/image/2023/06/06/202306060141536125_0.jpg
이미지가 성공적으로 다운로드되었습니다: ./마동석/마동석_3.jpg
https://images.khan.co.kr/article/2021/11/30/l_2021113002001725100347971.jpg
이미지가 성공적으로 다운로드되었습니다: ./마동석/마동석_4.jpg
https://i.namu.wiki/i/qGaWvOtSaMGOUrr5qyU_CiymXSeZeslEIAxlhRYU8a6DxBetExjgXlxYZdjIcwkvGWjCddGcyzUMOe_zLM2-bg.webp
이미지가 성공적으로 다운로드되었습니다: ./마동석/마동석_5.webp
https://www.newsflix.co.kr/news/photo/202210/10542_9844_055.png
이미지가 성공적으로 다운로드되었습니다: ./마동석/마동석_6.png
https://image.newsis.com/2023/07/09/NISI20230709_0001310466_web.jpg
이미지가 성